In [ ]:
#hide
%load_ext autoreload
%autoreload 2

# Welcome to gingado!

> A machine learning library for economics and finance

The purpose of `gingado` is to support usage of machine learning models in economics and finance use cases, promoting good modelling practices while being easy to use. `gingado` aims to be suitable for beginners and advanced users alike.

## Install

To install `gingado`, simply run the following code on the terminal:

`$ pip install gingado`

## Overview

`gingado` is built around three main functionalities:
* **data augmentation**, to add more data from official sources, improving the machine models being trained by the user;
* **automatic benchmark model**, to enable the user to assess their models against a reasonably well-performant model; and
* **support for model documentation**, to embed documentation and ethical considerations in the model development phase.

Each of these functionalities is illustrated below for a user trying to forecast GDP growth. Each step builds on top of the previous one, and they can be done with one line of code! But to highlight how `gingado` can benefit users in multiple ways, the brief walk-through below goes over them separtely. 

Before stepping in, let's import the necessary packages and data.

### Setup

This walk-through will use the [Jordà-Schularick-Taylor Macrohistory Database](https://www.macrohistory.net) on macroeconomics and finance as an example. As a preliminary step, let's import `gingado` and other necessary libraries, and proceed to download the data:

In [ ]:
import pandas as pd

JST_url = "http://data.macrohistory.net/JST/JSTdatasetR5.dta"
jst = pd.read_stata(JST_url, iterator=False)

jst.tail()

,year,country,iso,ifs,pop,rgdpmad,rgdppc,rconpc,gdp,iy,...,eq_capgain,eq_dp,eq_capgain_interp,eq_tr_interp,eq_dp_interp,bond_rate,eq_div_rtn,capital_tr,risky_tr,safe_tr
2659,2013.0,USA,USA,111,315820.328999,31571.993947,103.425299,101.892671,16784.851,0.192086,...,0.271035,0.019355,NaN,NaN,NaN,0.023508,0.024601,0.139843,0.212405,-0.065168
2660,2014.0,USA,USA,111,318106.646578,32113.618881,105.186253,104.113597,17527.258,0.196377,...,0.136350,0.019199,NaN,NaN,NaN,0.025408,0.021817,0.130063,0.132729,0.122398
2661,2015.0,USA,USA,111,320413.930388,32800.923063,107.421590,107.192931,18224.780,0.198301,...,-0.000092,0.021124,NaN,NaN,NaN,0.021358,0.021122,0.046193,0.065433,-0.008779
2662,2016.0,USA,USA,111,322705.239927,33078.508719,108.318698,109.333457,18715.040,0.195831,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2663,2017.0,USA,USA,111,324802.861426,33593.446309,110.013284,111.389150,19519.424,0.204547,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


For practical purposes, let's use for this database only the information from 1991 onwards:

In [ ]:
jst.drop(labels=['iso', 'ifs', 'crisisJST_old'], axis=1, inplace=True)
jst['year'] = pd.to_datetime(jst['year'], format='%Y')

start_year = '1991-01-01'
end_year = '2016-01-01'
jst = jst[jst['year'] >= start_year]

In [ ]:
jst.tail()

,year,country,pop,rgdpmad,rgdppc,rconpc,gdp,iy,cpi,ca,...,eq_capgain,eq_dp,eq_capgain_interp,eq_tr_interp,eq_dp_interp,bond_rate,eq_div_rtn,capital_tr,risky_tr,safe_tr
2659,2013-01-01,USA,315820.328999,31571.993947,103.425299,101.892671,16784.851,0.192086,173.067206,-426.197,...,0.271035,0.019355,NaN,NaN,NaN,0.023508,0.024601,0.139843,0.212405,-0.065168
2660,2014-01-01,USA,318106.646578,32113.618881,105.186253,104.113597,17527.258,0.196377,175.997979,-365.193,...,0.136350,0.019199,NaN,NaN,NaN,0.025408,0.021817,0.130063,0.132729,0.122398
2661,2015-01-01,USA,320413.930388,32800.923063,107.421590,107.192931,18224.780,0.198301,176.301162,-407.769,...,-0.000092,0.021124,NaN,NaN,NaN,0.021358,0.021122,0.046193,0.065433,-0.008779
2662,2016-01-01,USA,322705.239927,33078.508719,108.318698,109.333457,18715.040,0.195831,178.575038,-428.350,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2663,2017-01-01,USA,324802.861426,33593.446309,110.013284,111.389150,19519.424,0.204547,182.415361,-439.642,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data augmentation

As we can see, the dataset contains mainly information on the GDP of other countries. There are many ways in which this dataset can be complement by a richer set of variables. For example, for the same time periods we can add detailed information on trade, inflation, labour markets, foreign currency exchange rates, and many other series that would help machine learning models better form predictions. 

`gingado` helps this step by facilitating *data augmentation*, ie complementing the user dataset with compatible data. `gingado` only sources data from official data sources, namely statistical agencies, central banks, and other relevant authorities at the domestic or international level.

In particular, `gingado`'s function `augm_with_sdmx` looks for all data available from the desired sources for the defined period and frequency and downloads them all, but only adds to the original dataset the variables that bring some level of variance in the data that can be explored by the machine learning models later on.

In [ ]:
from gingado.augmentation import augm_with_sdmx

jst_augm_gingado = augm_with_sdmx(jst, start_year, end_year, time_col='year', freq='A', sources='BIS')

  6%|▌         | 1/18 [00:00<00:07,  2.23it/s]

Source: BIS, dataflow: WS_CBPOL_D ok!


 11%|█         | 2/18 [00:00<00:05,  2.89it/s]

Source: BIS, dataflow: WS_CBPOL_M ok!


 17%|█▋        | 3/18 [00:01<00:04,  3.06it/s]

...

Successful
Getting data from BIS's WS_XTD_DERIV


100%|██████████| 4/4 [00:10<00:00,  2.61s/it]


Successful


0it [00:00, ?it/s]


AttributeError: 'NoneType' object has no attribute 'dropna'

### Automatic benchmark

Now

## Design principles

The choices made during development of `gingado` derive from the following principles, in no particular order:
* *lowering the barrier to use machine learning* can help more economists familiarise themselves with these techniques and use them when appopriate
* *promoting good practices* such as documenting ethical considerations and benchmarking models as part of machine learning development will help embed these habits in economists
* *offering compatibility with other existing software that is consolidated by wide practice* benefits users and should be promoted as much as possible